# Input

In [396]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
import xgboost
import lightgbm as lgm
import catboost

In [397]:
train = pd.read_csv('train.csv' , parse_dates= ['date_created', 'user_date_created'] )
test = pd.read_csv('test.csv' ,parse_dates= ['date_created', 'user_date_created'] )

# Pre

In [398]:
test.drop(columns = 'month_created' , inplace = True)
train = train[list(test.columns) + ['is_fake']]

In [399]:
train.is_fake.value_counts()

False    1276801
True        4698
Name: is_fake, dtype: int64

In [400]:
train.shape , test.shape

((1281499, 11), (252099, 10))

In [401]:
(train['date_created'].max() - train['date_created'].min() ).days , 
(test['date_created'].max() - test['date_created'].min() ).days

145

In [402]:
train.head()

,comment,comment_id,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip,is_fake
0,компания не выполнила заказ без обьяснения пр...,1,2169053,2014-10-23 20:26:44,6.0,1,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2010-02-03 21:03:22,91933,178.92.70.220,False
1,"Впервые встретился такой професиональный, отве...",2,709306,2016-12-10 08:38:27,2.0,5,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,2010-02-03 21:03:22,91933,94.179.59.162,False
2,"Несерйозная, некоректная менеджер інтернет маг...",3,35287,2017-02-11 15:55:14,NaN,1,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,2010-02-03 21:03:22,91933,37.73.137.188,False
3,"Не рекомендую. Не отвечают, не выполняют заказы.",4,1778322,2017-06-26 18:00:45,1.0,1,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,2010-02-03 21:03:22,91933,37.73.149.180,False
4,"На письма не отвечают, заказы не выполняют. Не...",5,2821533,2017-07-06 16:57:40,NaN,1,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,2010-02-03 21:03:22,91933,37.73.221.126,False


In [403]:
test.head()

,comment,comment_id,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip
0,"спасибо все отлично, качество как на картинке,...",1655822,2214161,2018-03-19 21:36:07,1301549.0,5,NaN,2017-11-01 00:00:46,12969242,NaN
1,Заказывала шоколад Риттер Спорт 10 шт. К сожал...,1655823,2333068,2017-12-10 13:46:49,699700.0,5,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2017-11-01 00:02:19,12969248,91.90.13.210
2,"Быстро отзвонились, охотно отвечали на все воп...",1655824,752692,2017-11-01 00:07:38,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2017-11-01 00:02:44,12969251,93.115.95.204
3,Очень оперативный и внимательный к покупателю ...,1655825,334751,2017-11-20 23:40:01,1279247.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
4,Все прошло очень оперативно! Заказ привезли в ...,1655826,2263650,2017-11-20 23:39:51,1225545.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28


# Encoding

In [404]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat( [df[cat_cols] for df in df_list] , axis=0 )\
            .fillna('')\
            .apply(lambda x: d[x.name].fit(x)),
            
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(lambda x: d[x.name].transform(x))

__encode_categorical([train , test] , ['user_ip' , 'user_agent'])

In [405]:
startTime_date_created = train['date_created'].min()
startTime_user_created = train['user_date_created'].min()

train['date_created'] = [ (date - startTime_date_created).days  for date in train['date_created']]
test['date_created'] = [ (date - startTime_date_created).days  for date in test['date_created']]

train['user_date_created'] = [ (date - startTime_user_created).days  for date in train['user_date_created']]
test['user_date_created'] = [ (date - startTime_user_created).days  for date in test['user_date_created']]

train['is_fake'] = train['is_fake'].astype(int)

In [406]:
train.head()

,comment,comment_id,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip,is_fake
0,компания не выполнила заказ без обьяснения пр...,1,2169053,1709,6.0,1,62271,0,91933,150536,0
1,"Впервые встретился такой професиональный, отве...",2,709306,2487,2.0,5,30696,0,91933,745089,0
2,"Несерйозная, некоректная менеджер інтернет маг...",3,35287,2550,NaN,1,41360,0,91933,380703,0
3,"Не рекомендую. Не отвечают, не выполняют заказы.",4,1778322,2685,1.0,1,41363,0,91933,380968,0
4,"На письма не отвечают, заказы не выполняют. Не...",5,2821533,2695,NaN,1,41364,0,91933,385739,0


In [277]:
pp.ProfileReport(train)

Number of variables,11
Number of observations,1281499
Total Missing (%),3.2%
Total size in memory,102.7 MiB
Average record size in memory,84.0 B
Numeric,7
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,2


In [407]:
train = train.fillna(-1 )
test = test.fillna(-1 )

# Val

In [410]:
X = train.drop(columns = ['is_fake' , 'comment' , 'comment_id'])
test_X = test.drop(columns = [ 'comment' , 'comment_id'])
y = train['is_fake'].astype(int)

In [411]:
from sklearn.model_selection import StratifiedKFold , cross_val_score , GridSearchCV
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42 )

In [412]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=1/5, random_state=42 ,  stratify = y)

# Model

In [414]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
train_train_text = train.loc[X_train.index , 'comment']
train_test_text = train.loc[ X_test.index ,'comment']
train_text = train['comment']
test_text = test['comment']

all_text = pd.concat([train_text, test_text])

word_vectorizer = TfidfVectorizer( min_df = 5 , ngram_range = (1 , 1))
word_vectorizer.fit(all_text)

train_train_word = word_vectorizer.transform(train_train_text)
train_test_word = word_vectorizer.transform(train_test_text)
train_word = word_vectorizer.transform(train_text)
test_word = word_vectorizer.transform(test_text)

word_vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [415]:

classifier = LogisticRegression(C =100 , solver ='sag' , n_jobs =-1) 

classifier.fit(train_train_word , y_train)
log_loss(y_test , classifier.predict(train_test_word))

0.11724090912376309

In [416]:
from sklearn.linear_model import LogisticRegression
score = []
prediction = pd.Series( index = train.index )
lg = LogisticRegression(C =100 , solver ='sag' , n_jobs =-1) 

for train_val , test_val in skf.split(X ,y):
    train_features_tmp = word_vectorizer.transform(train.loc[train_val , 'comment'])
    test_features_tmp= word_vectorizer.transform(train.loc[test_val , 'comment'])
    
    lg.fit(train_features_tmp , y[train_val])
    pred = lg.predict(test_features_tmp)
    
    prediction[test_val] = lg.predict_proba(test_features_tmp)[:,1]
    score.append(log_loss(y[test_val] ,  pred))
    print(score[-1])

0.11670147637033024
0.11872332555720068
0.11899280022917025
0.11899334873669795
0.1137376299561838


In [418]:
classifier.fit(train_word , y)
test_word_pred =  classifier.predict_proba( test_word)[:,1]
test_word_pred.sum()

1021.9344502298575

In [422]:
X_st = X
X_st['pred'] = prediction
test_X_st = test_X
test_X_st['pred'] = test_word_pred
X_st.head()

,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip,pred
0,2169053,1709,6.0,1,62271,0,91933,150536,8.117127e-06
1,709306,2487,2.0,5,30696,0,91933,745089,1.739003e-05
2,35287,2550,-1.0,1,41360,0,91933,380703,1.698878e-06
3,1778322,2685,1.0,1,41363,0,91933,380968,3.240788e-07
4,2821533,2695,-1.0,1,41364,0,91933,385739,8.310157e-06


In [423]:
from sklearn.tree import DecisionTreeClassifier 
tree = DecisionTreeClassifier()

score = cross_val_score(tree , X_st , y ,cv = skf , scoring='neg_log_loss')
score.mean()

-0.06077643423297942

In [424]:
classifier.fit(X_st , y )
pred = classifier.predict_proba(test_X_st)

In [425]:
pred

array([[3.75359734e-01, 6.24640266e-01],
       [9.99564820e-01, 4.35180447e-04],
       [9.99341244e-01, 6.58756045e-04],
       ...,
       [9.92994837e-01, 7.00516315e-03],
       [9.94079311e-01, 5.92068887e-03],
       [9.84579596e-01, 1.54204037e-02]])

In [426]:
submisson =  pd.DataFrame( classifier.predict(test_X_st ) , index = test['comment_id'] , columns = ["is_fake"] )
submisson.to_csv('submisson_')